In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.svm import LinearSVC
import matplotlib
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

In [12]:
from useful_scripts import data_prepare
post, train=scripts.data_prepare.load_train_data()
info_types=scripts.data_prepare.load_label_map()

ModuleNotFoundError: No module named 'useful_scripts'

In [3]:
info_types

,type_id,description
type_name,,
bastard,0,These are mafia games where the moderator is a...
beginners-mafia,1,"Simple games with fixed rules, tailored toward..."
byor,2,"""Bring Your Own Role"": custom games where the ..."
classic,3,"In ""classic"" games, all players know what role..."
closed-setup,4,Closed setups mean that players do not know wh...
cybrid,5,"A custom, titled mafia game with multiple ""evi..."
kotm,6,"""King of the Mafia"": multiple mafia mini-games..."
non-mafia-game,7,Games which are sufficiently different from Ma...
other,8,"Non-game threads, such as discussion topics, s..."


In [4]:
post=pd.read_csv(r"E:\Nikita\Kaggle\Mafia gametype\train\post.csv")
thread=pd.read_csv(r"E:\Nikita\Kaggle\Mafia gametype\train\thread.csv")
post_test=pd.read_csv(r"E:\Nikita\Kaggle\Mafia gametype\test\post.csv")
thread_test=pd.read_csv(r"E:\Nikita\Kaggle\Mafia gametype\test\thread.csv")

In [5]:
post.head(3)

,thread_num,user,text,quotes
0,45016,Mephansteras,"Basically, this is where we talk about what ga...",[]
1,45016,dakarian,The currently running or about to run games (i...,[]
2,45016,webadict,And mine's started.\r\r\r\n\r\r\r\nI'll try to...,[]


In [6]:
thread.head(3)

,thread_num,thread_name,thread_label,thread_replies,thread_label_id
0,45016,Games Threshold Discussion and List [Vote for ...,other,5703,8
1,88720,New Player's Guide to the Subforum - New to Ma...,other,961,8
2,39338,Mafia: A Basic Tutorial,other,79,8


In [7]:
import re
def clean(raw_data):
    review_text = BeautifulSoup(raw_data).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))  

In [8]:
num=len(thread)
data_to_clean=[]
clean_test_data=[]
test_data_to_clean=[]

In [9]:
for item in thread.itertuples():
    buf=post[post["thread_num"]==item.thread_num]
    line=str()
    line=line+item.thread_name
    for row in buf.itertuples():
        line=line+str(row.text)
    data_to_clean.append(line)
    
    
    
    
for item in thread_test.itertuples():
    buf=post_test[post_test["thread_num"]==item.thread_num]
    line=str()
    line=line+item.thread_name
    for row in buf.itertuples():
        line=line+str(row.text)
    test_data_to_clean.append(line)

358

97539

In [10]:
from bs4 import BeautifulSoup

In [11]:
clean_data = []
for item in data_to_clean:
    clean_data.append(clean(item))
    
    
for item in test_data_to_clean:
    clean_test_data.append(clean(item))

Cleaning and parsing the training set movie reviews...

Cleaning and parsing the training set movie reviews...



In [12]:
X_train=clean_data
y_train=thread["thread_label_id"]
#X_train, X_val, y_train, y_val = train_test_split(clean_data, thread["thread_label_id"], test_size=0.2, random_state=35)

In [13]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,  preprocessor = None,
                             stop_words = None, max_features = 1500) 

train_data_features = vectorizer.fit_transform(X_train)

train_data_features = train_data_features.toarray()



test_data_features = vectorizer.transform(clean_test_data)

test_data_features = test_data_features.toarray()



#val_data_features = vectorizer.fit_transform(X_val)
#word_df = pd.DataFrame(train_data_features, columns=vectorizer.get_feature_names(), index=thread.index)

#val_data_features = val_data_features.toarray()

#train_data_features = pd.concat([(thread['thread_replies'] + 1).rename('posts'), word_df,], axis='columns')
#train_data_features.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 



forest = forest.fit( train_data_features, y_train)

val_data = forest.predict(val_data_features)

from sklearn.metrics import accuracy_score

acc = accuracy_score(y_val, val_data)

print("Accuracy on the IMDB dataset: {:.2f}".format(acc*100))

In [184]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())

classifier.fit(train_data_features, y_train.as_matrix())

predictions = classifier.predict(val_data_features)
# accuracy
print("Accuracy = ",accuracy_score(y_val,predictions.A))
print("\n")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy =  0.5694444444444444




In [17]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())

classifier.fit(train_data_features, y_train.as_matrix())

predictions = classifier.predict(val_data_features)

print("Accuracy = ",accuracy_score(y_val,predictions.A))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


BinaryRelevance(classifier=GaussianNB(priors=None, var_smoothing=1e-09),
        require_dense=[True, True])

In [187]:
model = LinearSVC()
model.fit(train_data_features, y_train)
y_pred = model.predict(val_data_features)

# Evaluation
print(accuracy_score(y_pred, y_val))

0.5972222222222222


Cleaning and parsing the training set movie reviews...



In [20]:
#test_data_features = vectorizer.transform(clean_test_data)

#test_data_features = test_data_features.toarray()

#test_df = pd.DataFrame(train_data_features, columns=vectorizer.get_feature_names(), index=thread.index)

#test_data_features = pd.concat([(thread['thread_replies'] + 1).rename('posts'), test_df,], axis='columns')

NameError: name 'forest' is not defined

In [21]:
test_data_features.head()

,posts,absence,accepting,active,activity,actually,age,agent,aka,alien,...,xxiii,xxiv,xxv,ya,yes,yet,yo,zathras,zerzooton,zombie
0,5704,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,962,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,80,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,308,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
from sklearn.tree import DecisionTreeClassifier

# Just using default parameters... what can do wrong?
cls = DecisionTreeClassifier(random_state=1000)
cls.fit(train_data_features, y_train)
third = cls.predict(test_data_features)

In [15]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 


forest = forest.fit( train_data_features, y_train)

first = forest.predict(test_data_features)



In [18]:
lab=pd.Series(first)
ans = pd.concat([thread_test["thread_num"],lab], axis=1, keys=['thread_num', 'thread_label_id'])
ans=ans.set_index("thread_num")
ans.head()

,thread_label_id
thread_num,
126856,8
132415,1
134482,8
133728,1
134270,8


In [16]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())

classifier.fit(train_data_features, y_train.as_matrix())
second = classifier.predict(test_data_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [ ]:
print(second)

In [ ]:
second=second.data
second

In [28]:
label=pd.Series(second,dtype=np.int32)
ans = pd.concat([thread_test["thread_num"],label], axis=1, keys=['thread_num', 'thread_label_id'])
ans=ans.set_index("thread_num")
ans.head()

,thread_label_id
thread_num,
126856,3.0
132415,8.0
134482,8.0
133728,1.0
134270,3.0


In [39]:
ans.to_csv(r"E:\Nikita\Kaggle\Mafia gametype\subm\sol6.csv")

In [27]:
print(third)

NameError: name 'third' is not defined

In [46]:
lab=pd.Series(third)
ans = pd.concat([thread_test["thread_num"],lab], axis=1, keys=['thread_num', 'thread_label_id'])
ans=ans.set_index("thread_num")
ans.head()

,thread_label_id
thread_num,
126856,12
132415,12
134482,12
133728,11
134270,8


In [19]:
ans.to_csv(r"E:\Nikita\Kaggle\Mafia gametype\subm\sol10.csv")

In [17]:
print(first)
print(second.data)
print(third)

[ 8  1  8  1  8  1  4  8  9  2  1  8  1  8  8  2  8  1  2  8 10  1  8  8
  1  8  8  2  8  2  2  8  8  8  1  8  2  2  4  8  1  9  4  1  8  4  2  8
  8  8  2 10  1  4  8  2  1  9  4  8  1  2  8  8  1  2  8  1  8  4  9  1
  8  0  4  2  2  4  1  9  8  8  8  3  2  2  1  8  8  8  4  0  1  2  8  8
  1  1  8  4  4  1  2  8  8  8  8  4  4  4  4  8  8  8  8  8  2  1  8  8
  8  2  8  0  8  2  2  2  8  2  8  4  2  1  8  2  8  8  8  9  8  2  2  2
  8  1  8  8  3  4  4  8  1  4  8  2  2  8  8  8  4  1  2  1  8  8  8  4
  8  8  1  8  4  8  1  2  2  8  8  1  8  8  8  8  4  4  1  8  4  8  1  4
  9  1  2  1  1  8  1  1  1  2  2  8  2  1  8  2  1  8  1  8  8  2  1  8
  8  8  4  8  8  8  1  8  4  4  8  4  4 10  8  8  4  3  2  8]
[ 3  8  8  1  3  8  4  3  9  8  1 11  8  8  8  8  3  8  8  3  4  8  8  3
  8  8  3  8  3  3  8  3  3  8  8  3  8  8  8  3  8  8  8  1  3  8  8  3
  3  8  8  8  1  8  3  8  8  8  8  3  1  8  8  8  1  4  8  8  8  8  8  8
  3  8  8  8  8  3  8  9  3  8  8  8  8  3  8  3  3  8  8  8  